In [3]:
import sys, json, os
from itertools import permutations
from multiprocessing import Pool

os.chdir('/data/data1/conglab/qcong/CASP16/oligomer_inputs/')


target = "H0208"
name = "H0208"

fp = open('unknown_stoic/' + target + '/' + name + '.txt', 'r')
start = 0
models = []
for line in fp:
    words = line.split()
    if words:
        if words[0] == '#':
            start = 1
        elif start and len(words) > 1:
            models.append(words[1])
fp.close()

fp = open('unknown_stoic/' + target + '/' + name + '.pdb', 'r')
Rchain2resids = {}
for line in fp:
    if len(line) > 60:
        if line[:4] == 'ATOM':
            resid = int(line[22:26])
            chain = line[21]
            try:
                Rchain2resids[chain].add(resid)
            except KeyError:
                Rchain2resids[chain] = set([resid])
fp.close()

bad_models = set(['T1269v1TS456_4o','T1269v1TS022_4o','T1269v1TS052_4o','T1269v1TS456_1o','T1269v1TS022_2o','T1269v1TS052_2o'])
bad2alt = {}
bad2alt['T1269v1TS022_2o'] = 'T1269v1TS022_1o'
bad2alt['T1269v1TS022_4o'] = 'T1269v1TS022_1o'
bad2alt['T1269v1TS052_2o'] = 'T1269v1TS052_1o'
bad2alt['T1269v1TS052_4o'] = 'T1269v1TS052_1o'
bad2alt['T1269v1TS456_1o'] = 'T1269v1TS456_2o'
bad2alt['T1269v1TS456_4o'] = 'T1269v1TS456_2o'
Rchain2resids

{'A': {17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  161,
  162,
  163,
  164,
  165,
  166,
  167,
  168,
  169,
  170,


In [ ]:
def check_model(model):
    with open('unknown_stoic/' + target + '/' + model + '.json', 'r') as file:
        data = json.load(file)
    if model in bad_models:
        alt_model = bad2alt[model]
        with open('unknown_stoic/' + target + '/' + alt_model + '.json', 'r') as file:
            alt_data = json.load(file)
    ref_chem_groups = data["chem_groups"]
    mod_chem_groups = data["chem_mapping"]

    assert len(ref_chem_groups) == len(mod_chem_groups)
    # print(ref_chem_groups)
    # print(mod_chem_groups)
    # breakpoint()

    if target[0] == 'T':
        groups = [[]]
        for i in range(len(ref_chem_groups)):
            if ref_chem_groups[i]:
                for chain in ref_chem_groups[i]:
                    groups[0].append(['ref', chain])
            if mod_chem_groups[i]:
                for chain in mod_chem_groups[i]:
                    groups[0].append(['mod', chain])

    else:
        groups = []
        for i in range(len(ref_chem_groups)):
            groups.append([])
            if ref_chem_groups[i]:
                for chain in ref_chem_groups[i]:
                    groups[i].append(['ref', chain])
            if mod_chem_groups[i]:
                for chain in mod_chem_groups[i]:
                    groups[i].append(['mod', chain])

    # print(groups)
    # breakpoint()
    
    Rchain_pairs = set([])
    Rpair2contacts = {}
    if model in bad_models:
        reference_contacts = alt_data['reference_contacts']
    else:
        reference_contacts = data['reference_contacts']
    # print(reference_contacts)
    # breakpoint()

    for refcon in reference_contacts:
        chain1 = refcon[0].split('.')[0]
        resid1 = int(refcon[0].split('.')[1])
        chain2 = refcon[1].split('.')[0]
        resid2 = int(refcon[1].split('.')[1])
        if chain1 < chain2:
            Rchain_pairs.add((chain1, chain2))
        elif chain2 < chain1:
            Rchain_pairs.add((chain2, chain1))
        try:
            Rpair2contacts[(chain1, chain2)].add((resid1, resid2))
        except KeyError:
            Rpair2contacts[(chain1, chain2)] = set([(resid1, resid2)])
        try:
            Rpair2contacts[(chain2, chain1)].add((resid2, resid1))
        except KeyError:
            Rpair2contacts[(chain2, chain1)] = set([(resid2, resid1)])
    # print(Rchain_pairs)
    # print(Rpair2contacts)


    Mchain_pairs = set([])
    Mpair2contacts = {}
    for modcon in data['model_contacts']:
        chain1 = modcon[0].split('.')[0]
        resid1 = int(modcon[0].split('.')[1])
        chain2 = modcon[1].split('.')[0]
        resid2 = int(modcon[1].split('.')[1])
        if chain1 < chain2:
            Mchain_pairs.add((chain1, chain2))
        elif chain2 < chain1:
            Mchain_pairs.add((chain2, chain1))
        try:
            Mpair2contacts[(chain1, chain2)].add((resid1, resid2))
        except KeyError:
            Mpair2contacts[(chain1, chain2)] = set([(resid1, resid2)])
        try:
            Mpair2contacts[(chain2, chain1)].add((resid2, resid1))
        except KeyError:
            Mpair2contacts[(chain2, chain1)] = set([(resid2, resid1)])
    # print(Mchain_pairs)
    # print(Mpair2contacts)
    # breakpoint()

    get_Rchains = set([])
    get_Mchains = set([])
    Rchain2Mchain = {}
    Mchain2Rchain = {}
    # print(groups)
    for group in groups:
        refchains = []
        modchains = []
        for item in group:
            if item[0] == 'ref':
                refchains.append(item[1])
            elif item[0] == 'mod':
                modchains.append(item[1])
        # print(refchains)
        for refchain in refchains:
            for modchain in modchains:
                try:
                    Rchain2Mchain[refchain].add(modchain)
                except KeyError:
                    get_Rchains.add(refchain)
                    Rchain2Mchain[refchain] = set([modchain])
                try:
                    Mchain2Rchain[modchain].add(refchain)
                except KeyError:
                    get_Mchains.add(modchain)
                    Mchain2Rchain[modchain] = set([refchain])
    # print(Rchain2Mchain)
    # print(Mchain2Rchain)
    all_results = []
    for Rchain_pair in Rchain_pairs:
        Rcontacts = Rpair2contacts[Rchain_pair]
        # print(Rchain_pair)
        # print(Rcontacts)
        Rresid1s = set([])
        Rresid2s = set([])
        for Rcontact in Rcontacts:
            Rresid1s.add(Rcontact[0])
            Rresid2s.add(Rcontact[1])
        Rchain1 = Rchain_pair[0]
        Rchain2 = Rchain_pair[1]
        # print(Rchain1, Rchain2)
        map_results = []
        if Rchain1 in get_Rchains and Rchain2 in get_Rchains:
            for Mchain1 in Rchain2Mchain[Rchain1]:
                for Mchain2 in Rchain2Mchain[Rchain2]:
                    try:
                        Mcontacts = Mpair2contacts[(Mchain1, Mchain2)]
                    except KeyError:
                        Mcontacts = set([])
                    # print(Mcontacts)
                    good_Mcontacts = set([])
                    good_Mresid1s = set([])
                    good_Mresid2s = set([])
                    print(Mcontacts)
                    for Mcontact in Mcontacts:
                        Mresid1 = Mcontact[0]
                        Mresid2 = Mcontact[1]
                        if Mresid1 in Rchain2resids[Rchain1] and Mresid2 in Rchain2resids[Rchain2]:
                            good_Mcontacts.add(Mcontact)
                            good_Mresid1s.add(Mresid1)
                            good_Mresid2s.add(Mresid2)
                        # else:
                        #     print(Mresid1, Mresid2)
                    # print(good_Mcontacts)
                    # print(good_Mresid1s)
                    # print(good_Mresid2s)

                    if good_Mcontacts:
                        Rcount = len(Rresid1s) + len(Rresid2s)
                        Mcount = len(good_Mresid1s) + len(good_Mresid2s)
                        RMcount = len(Rresid1s.intersection(good_Mresid1s)) + len(Rresid2s.intersection(good_Mresid2s))
                        ips = round(RMcount / (Rcount + Mcount - RMcount), 4)
                        overlap = Rcontacts.intersection(good_Mcontacts)
                        if overlap:
                            precision = len(overlap) / len(Rcontacts)
                            recall = len(overlap) / len(good_Mcontacts)
                            ics = round(2 * precision * recall / (precision + recall), 4)
                            qs_best = round(len(overlap) / max(len(Rcontacts), len(good_Mcontacts)), 4)
                        else:
                            ics = 0.0
                            qs_best = 0.0
                        map_results.append([Mchain1, Mchain2, len(Rresid1s), len(Rresid2s), ips, ics, qs_best])
                    else:
                        map_results.append([Mchain1, Mchain2, len(Rresid1s), len(Rresid2s), 0.0, 0.0, 0.0])
        all_results.append(['reference', Rchain1, Rchain2, len(Rresid1s), len(Rresid2s), map_results])

    for Mchain_pair in Mchain_pairs:
        Mcontacts = Mpair2contacts[Mchain_pair]
        Mresid1s = set([])
        Mresid2s = set([])
        for Mcontact in Mcontacts:
            Mresid1s.add(Mcontact[0])
            Mresid2s.add(Mcontact[1])
        Mchain1 = Mchain_pair[0]
        Mchain2 = Mchain_pair[1]

        map_results = []
        if Mchain1 in get_Mchains and Mchain2 in get_Mchains:
            for Rchain1 in Mchain2Rchain[Mchain1]:
                for Rchain2 in Mchain2Rchain[Mchain2]:
                    try:
                        Rcontacts = Rpair2contacts[(Rchain1, Rchain2)]
                    except KeyError:
                        Rcontacts = set([])
                    Rresid1s = set([])
                    Rresid2s = set([])
                    for Rcontact in Rcontacts:
                        Rresid1s.add(Rcontact[0])
                        Rresid2s.add(Rcontact[1])

                    good_Mcontacts = set([])
                    good_Mresid1s = set([])
                    good_Mresid2s = set([])
                    for Mcontact in Mcontacts:
                        Mresid1 = Mcontact[0]
                        Mresid2 = Mcontact[1]
                        if Mresid1 in Rchain2resids[Rchain1] and Mresid2 in Rchain2resids[Rchain2]:
                            good_Mcontacts.add(Mcontact)
                            good_Mresid1s.add(Mresid1)
                            good_Mresid2s.add(Mresid2)

                    if Rcontacts and good_Mcontacts:
                        Rcount = len(Rresid1s) + len(Rresid2s)
                        Mcount = len(good_Mresid1s) + len(good_Mresid2s)
                        RMcount = len(Rresid1s.intersection(good_Mresid1s)) + len(Rresid2s.intersection(good_Mresid2s))
                        ips = round(RMcount / (Rcount + Mcount - RMcount), 4)
                        overlap = Rcontacts.intersection(good_Mcontacts)
                        if overlap:
                            precision = len(overlap) / len(Rcontacts)
                            recall = len(overlap) / len(good_Mcontacts)
                            ics = round(2 * precision * recall / (precision + recall), 4)
                            qs_best = round(len(overlap) / max(len(Rcontacts), len(good_Mcontacts)), 4)
                        else:
                            ics = 0.0
                            qs_best = 0.0
                        map_results.append([Rchain1, Rchain2, len(good_Mresid1s), len(good_Mresid2s), ips, ics, qs_best])
                    else:
                        map_results.append([Rchain1, Rchain2, len(good_Mresid1s), len(good_Mresid2s), 0.0, 0.0, 0.0])
        all_results.append(['prediction', Mchain1, Mchain2, len(Mresid1s), len(Mresid2s), map_results])
    rp = open('step7/' + target + '/' + model + '.result','w')
    for result in all_results:
        category = result[0]
        chain1 = result[1]
        chain2 = result[2]
        size1 = str(result[3])
        size2 = str(result[4])
        rp.write('>' + category + ' ' + chain1 + ':' + chain2 + ' ' + size1 + ':' + size2 + '\n')
        for item in result[5]:
            rp.write(item[0] + '\t' + item[1] + '\t' + str(item[2]) + '\t' + str(item[3]) + '\t' + str(item[4]) + '\t' + str(item[5]) + '\t' + str(item[6]) + '\n')
    rp.close()
    return 0

count = 0
for model in models:
    check_model(model)
    count += 1
    if count >= 25:
        break

set()
{(197, 269), (299, 36), (103, 280), (24, 225), (104, 279), (305, 150), (308, 112), (289, 269), (22, 282), (282, 239), (306, 297), (299, 84), (298, 9), (303, 179), (32, 231), (132, 288), (311, 3), (104, 281), (302, 9), (285, 240), (133, 280), (199, 280), (73, 238), (128, 293), (31, 232), (313, 3), (23, 228), (309, 298), (197, 273), (129, 285), (32, 279), (258, 204), (299, 86), (298, 11), (132, 290), (30, 233), (311, 5), (256, 14), (25, 228), (308, 299), (73, 231), (302, 84), (22, 277), (31, 234), (313, 5), (261, 269), (301, 113), (308, 109), (261, 241), (305, 284), (310, 298), (300, 178), (257, 173), (23, 278), (252, 12), (23, 232), (105, 280), (259, 173), (132, 285), (289, 241), (104, 278), (25, 232), (103, 282), (22, 281), (29, 232), (252, 14), (102, 280), (168, 280), (296, 176), (196, 272), (311, 2), (254, 14), (25, 225), (308, 113), (304, 109), (31, 231), (305, 108), (133, 282), (296, 178), (197, 272), (305, 300), (299, 85), (300, 84), (30, 232), (311, 4), (200, 283), (128, 28